In [1]:
import os
import numpy as np 
import pandas as pd 

In [3]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/stanford_politeness_wiki_furthest.csv

--2022-12-21 10:33:28--  https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/stanford_politeness_wiki_furthest.csv
Resolving cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)... 52.216.211.65, 3.5.11.209, 52.216.178.219, ...
Connecting to cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)|52.216.211.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196435 (192K) [text/csv]
Saving to: './../../../dataset/stanford_politeness_wiki_furthest.csv'

stanford_politeness 100%[===================>] 191.83K   548KB/s    in 0.3s    

2022-12-21 10:33:46 (548 KB/s) - './../../../dataset/stanford_politeness_wiki_furthest.csv' saved [196435/196435]



In [4]:
path = "./../../../dataset/stanford_politeness_wiki_furthest.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (1311, 5)


,text,meta.Binary,true_label,given_label,true_error
0,"I can't spend too much time, and I'm no specia...",0,17.0,11.0,0
1,<url>. I wonder if anyone will actually suspec...,0,13.0,13.0,0
2,"Ok, that's no problem. Can you recommend any o...",1,17.0,20.0,0
3,"Well, that was an example as I am adding Infob...",0,13.0,20.0,1
4,"Hi, i saw in a lot of biography articles are u...",0,17.0,9.0,0


In [5]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,text,meta.Binary,given_label
0,"I can't spend too much time, and I'm no specia...",0,11.0
1,<url>. I wonder if anyone will actually suspec...,0,13.0


In [6]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [7]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [8]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    1311
Train Data Columns: 2
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1319.42 MB
	Train Data (Original)  Memory Usage: 0.26 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator..

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.782212,0.295220,0.033516,0.021059,0.783403,0.033516,0.021059,0.783403,1,True,2
1,WeightedEnsemble_L2,0.607446,0.323534,0.048302,0.031636,2.237615,0.002497,0.000261,0.031238,2,True,4
2,LightGBM,0.497689,0.278344,0.005088,0.002148,1.438684,0.005088,0.002148,1.438684,1,True,1
3,NeuralNetFastAI,0.449496,0.315095,0.012289,0.010317,1.422973,0.012289,0.010317,1.422973,1,True,3


### Cross Validation fitting

In [9]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    1311
Train Data Columns: 2
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1321.67 MB
	Train Data (Original)  Memory Usage: 0.26 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerat

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.909265,0.340596,0.048730,0.058147,0.620326,0.048730,0.058147,0.620326,1,True,2
1,WeightedEnsemble_L2,0.669053,0.367674,0.188345,0.107723,9.694040,0.001185,0.000167,0.033570,2,True,4
2,LightGBM_BAG_L1,0.590667,0.340268,0.044289,0.014684,4.689304,0.044289,0.014684,4.689304,1,True,1
3,NeuralNetFastAI_BAG_L1,0.493886,0.341153,0.094141,0.034725,4.350840,0.094141,0.034725,4.350840,1,True,3


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))